In [3]:
import numpy as np
import pandas as pd
import nltk
import sys
import re
# import Levenshtein

In [89]:
corpus_v2 = pd.read_csv("majorApp_expertsentence.csv")

In [90]:
corpus_v2

,sentence,task_id
0,What is the population of Arkansas on July 1st...,1
1,Where is Harvard University located?,1
2,How many faculties are there in Harvard Univer...,1
3,What type of school is Harvard University?,1
4,"Which county has more than 30,000 people?",2
5,Which university is in New York?,2
6,Which university has more than 30 faculties an...,2
7,"Which county has less than 2,000 people in 200...",2
8,What is the mean population in 2000?,3
9,What is the mean population of Ashley in 2000?,3


In [17]:
len(corpus_v2[(corpus_v2.Task == '1') | (corpus_v2.Task == 1) | corpus_v2.Task.str.contains('1,', case=False, na=False)])

93

In [91]:
corpus_v2.iloc[818]

sentence    Where is Barcelona – Sants?
task_id                               1
Name: 818, dtype: object

In [20]:
corpus_v2[corpus_v2.task_id==1].sentence

0      What is the population of Arkansas on July 1st...
1                   Where is Harvard University located?
2      How many faculties are there in Harvard Univer...
3             What type of school is Harvard University?
42     What is the ranking of Iceland in the World Ha...
43     How many dimensions describe the happiness score?
44     How many students have had their grades recorded?
45                  Indicate Kibo Mcgee's midterm score.
83                  How much revenue did "Titanic" earn?
84       How many goals did Liverpool score in the game?
85     Find the score that Jack received in the final...
86     How long would it take to arrive home from sch...
127                  What region does Iceland belong to?
128                   What is the 15th happiest country?
129    How would people in Iceland rate their happiness?
130    If Iceland wanted to enhance its happiness sco...
154    Which publisher published "The Queen of Nothing"?
155          Enumerate the book

In [17]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [127]:
sents = "I'm xiong kai-, how (are you? gg;)"
re.sub('[?,."()\':;–-]','',sents).lower()

'im xiong kai how are you gg'

In [128]:
def preprocess_sentences(sentences):
    new_sents = []
    for sentence in list(sentences):
        # remove punctuation and ignore cases
        new_sents.append(re.sub('[?,."()\':;–-]','',sentence).lower())
    return new_sents

In [33]:
def sentences_group_by_task(corpus):
    tasks_sentences = {}
    for i in range(1,11):
        tasks_sentences[i] = preprocess_sentences(corpus[corpus.task_id==i].sentence)
        
    return tasks_sentences

In [129]:
tasks_sentences = sentences_group_by_task(corpus_v2)

In [114]:
tasks_sentences[1][:2]

['what is the population of arkansas on july 1st 2002',
 'where is harvard university located']

In [115]:
tasks_sentences[10][:2]

['is the population in 2000 related to that in 2001',
 'does the number of teachers affect the number of faculties']

In [38]:
len(tasks_sentences[1])

93

In [95]:
hypothesis = ["open", "the", "file"]
reference = ["please", "open", "this","file"]
#the maximum is bigram, so assign the weight into 2 half.
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, weights = (0.5, 0.5))
print(BLEUscore)

8.726935353908751e-155


d:\software\study\python37\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [96]:
print(set(nltk.ngrams(["reference", "ha"],2)))

{('reference', 'ha')}


In [116]:
class Indices(object):
    def __init__(self):
        self.unigram = {}
        self.bigram = {}
        self.trigram = {}
        self.sentence_len = []

        self.levenshtein = []
        self.bleu_score = []
        
        self.uni_overlap = []
        self.bi_overlap = []
        self.tri_overlap = []

        self.uni_set = []
        self.bi_set = []
        self.tri_set = []
        
    
    # caluclate the ngram of the words
    #
    # @param    words       list{str}
    # @return   none
    def ngram(self, words):
        wlen = len(words)
                  
        for i in range(wlen):
            word = words[i]
                  
            if word not in self.unigram:
                self.unigram[word] = 1
            else:
                self.unigram[word] = self.unigram[word] + 1
            
            if i < (wlen - 1):
                bi_words = ' '.join([word, words[i+1]])
                if bi_words not in self.bigram:
                    self.bigram[bi_words] = 1
                else:
                    self.bigram[bi_words] = self.bigram[bi_words] + 1
                
                if i < (wlen - 2):
                    tri_words = ' '.join([word, words[i+1], words[i+2]])
                    if tri_words not in self.trigram:
                        self.trigram[tri_words] = 1
                    else:
                        self.trigram[tri_words] = self.trigram[tri_words] + 1
                        
        
    def lev_dist(self, source, target):
        '''
            Compute Levenshtein Distance, Edit Distance
        '''
        len_source = len(source)
        len_target = len(target)
        edit = [[i + j for j in range(len_target + 1)] for i in range(len_source + 1)]
        for i in range(1, len_source + 1):
            for j in range(1, len_target + 1):
                if source[i - 1] == target[j - 1]:
                    d = 0
                else:
                    d = 1
                edit[i][j] = min(edit[i - 1][j] + 1, edit[i][j - 1] + 1, edit[i - 1][j - 1] + d)

        max_len = max(len_source, len_target)
        edit_score = edit[len_source][len_target]
        return (max_len - edit_score)/ max_len


    def overlap(self, s_num):
        for si in range(s_num - 1):
            for sj in range(si + 1, s_num):
                try:
                    self.uni_overlap.append(len(self.uni_set[si] & self.uni_set[sj]) / len(self.uni_set[si] | self.uni_set[sj]))
                    self.bi_overlap.append(len(self.bi_set[si] & self.bi_set[sj]) / len(self.bi_set[si] | self.bi_set[sj]))
                    self.tri_overlap.append(len(self.tri_set[si] & self.tri_set[sj]) / len(self.tri_set[si] | self.tri_set[sj]))
                except Exception as e:
                    print(self.uni_set[si], self.uni_set[sj])
                    print(self.bi_set[si], self.bi_set[sj])
                    print(self.tri_set[si], self.tri_set[sj])

In [81]:
import json

In [131]:
result = {}
for taski, ri in analyse_corpus.result.items():
    result[taski] = {"unigram": ri.unigram, "bigram": ri.bigram, "trigram": ri.trigram, "sentences_len": ri.sentence_len, 
                     "levenshtein": ri.levenshtein, "bleu_score": ri.bleu_score, 
                     "uni_overlap": ri.uni_overlap, "bi_overlap": ri.bi_overlap, "tri_overlap": ri.tri_overlap}
with open('./CorpusAnalysisResult2.json','w',encoding='utf-8') as fObj:
    json.dump(result,fObj,ensure_ascii=False)

In [76]:
tasks_sentences[1][0].split()

['what',
 'is',
 'the',
 'population',
 'of',
 'arkansas',
 'on',
 'july',
 '1st',
 '2002']

In [117]:
class AnalyseCorpus(object):

    def __init__(self, corpus):
        self.corpus = corpus  # corpus: {1:[], 2:[]}
        self.result = {}


    def analyse(self):
        for taski, ci in self.corpus.items():  # 每一个task的sentences：ci
            indices = Indices()
            
            s_num = len(ci)   # 此task下共有多少句sentence
            for si in range(s_num):
                words = ci[si].split()
                indices.sentence_len.append(len(words))

                indices.uni_set.append(set(nltk.ngrams(words,1)))
                indices.bi_set.append(set(nltk.ngrams(words,2)))
                indices.tri_set.append(set(nltk.ngrams(words,3)))

                indices.ngram(words)
                if si < s_num - 1:
                    for sj in range(si + 1, s_num):
                        words2 = ci[sj].split()
                        indices.levenshtein.append(indices.lev_dist(words, words2))
                        indices.bleu_score.append(nltk.translate.bleu_score.sentence_bleu([words], words2, weights = (0.5, 0.5)))

            indices.overlap(s_num)
            self.result[taski] = indices
        
        '''
        # unigram
        
        try:
            fip = open(self.name + "uni", "w", encoding='utf-8')
        except:
            print(sys.stderr, "failed to open data.uni")
        tmp = zip(self.unigram.values(),self.unigram.keys())
        for i in sorted(tmp,reverse=True):
            fip.write("%s %d\n" % (i[1], i[0]))
            
        # bigram
        try:
            fip = open(self.name + "bi", "w", encoding='utf-8')
        except:
            print(sys.stderr, "failed to open data.bi")
        tmp = zip(self.bigram.values(),self.bigram.keys())
        for i in sorted(tmp,reverse=True):
            fip.write("%s %d\n" % (i[1], i[0]))
            
        # trigram
        try:
            fip = open(self.name + "tri", "w", encoding='utf-8')
        except:
            print(sys.stderr, "failed to open data.tri")
        tmp = zip(self.trigram.values(),self.trigram.keys())
        for i in sorted(tmp,reverse=True):
            fip.write("%s %d\n" % (i[1], i[0]))
        
        try:
            fip = open(self.name + "len", "w", encoding='utf-8')
            fip.write("Sentences Num: %d\n" % (len(self.sentence_len)))
        except:
            print(sys.stderr, "failed to open data.len")
        for i in self.sentence_len:
            fip.write("%d, " % (i))
            
        '''
            

In [130]:
analyse_corpus = AnalyseCorpus(tasks_sentences)
analyse_corpus.analyse()

In [119]:
print(len(analyse_corpus.result[1].levenshtein))
analyse_corpus.result[1].levenshtein[:10]

4278


[0.1, 0.0, 0.2, 0.36363636363636365, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [52]:
print(len(ngram.levenshtein))
ngram.levenshtein[:10]

4278


[9, 10, 6, 7, 10, 10, 10, 6, 10, 8]

In [120]:
print(len(analyse_corpus.result[1].uni_overlap))
analyse_corpus.result[1].uni_overlap[:10]

4278


[0.07142857142857142,
 0.0,
 0.21428571428571427,
 0.25,
 0.0625,
 0.0,
 0.0,
 0.0,
 0.05555555555555555,
 0.05555555555555555]

In [54]:
print(len(ngram.uni_overlap))
ngram.uni_overlap[:10], ngram.bi_overlap[:10]

4278


([0.07692307692307693,
  0.0,
  0.3076923076923077,
  0.3076923076923077,
  0.0,
  0.0625,
  0.0,
  0.3333333333333333,
  0.05555555555555555,
  0.125],
 [0.0,
  0.0,
  0.15384615384615385,
  0.15384615384615385,
  0.0,
  0.0,
  0.0,
  0.16666666666666666,
  0.0,
  0.0])

In [57]:
ngram.sentence_len[:10]

[10, 4, 8, 7, 7, 7, 7, 6, 6, 9]

In [ ]:
print(Levenshtein.distance(string1, string2))

In [59]:
ref = set(nltk.ngrams("What is the popularity of Arkansas on July 1st 2002".split(),1))
hyp = set(nltk.ngrams("What is the happiness rank of Iceland".split(),1))
len(hyp&ref) / len(hyp|ref)

0.3076923076923077

In [9]:
import pandas as pd
import json

In [3]:
table = pd.read_csv(r"D:\lab_data\之江lab\AIforVis\main\static\table\movies.csv")

In [20]:
list(table["Revenue"])

[2781505847,
 1835300000,
 1327655619,
 1062984497,
 1023285206,
 1001921825,
 919700000,
 892194397,
 887773705,
 885430303,
 880871036,
 866300000,
 865000000,
 836276689,
 817068851]

In [21]:
aa = table[["Release Year","Revenue"]][:3]

In [27]:
aa

,Release Year,Revenue
0,2009,2781505847
1,1997,1835300000
2,2011,1327655619


In [28]:
aa.to_json(orient="columns")  # 默认是columns

'{"Release Year":{"0":2009,"1":1997,"2":2011},"Revenue":{"0":2781505847,"1":1835300000,"2":1327655619}}'

In [23]:
aa.to_json(orient="split")

'{"columns":["Release Year","Revenue"],"index":[0,1,2],"data":[[2009,2781505847],[1997,1835300000],[2011,1327655619]]}'

In [24]:
aa.to_json(orient="records")

'[{"Release Year":2009,"Revenue":2781505847},{"Release Year":1997,"Revenue":1835300000},{"Release Year":2011,"Revenue":1327655619}]'

In [25]:
aa.to_json(orient="index")

'{"0":{"Release Year":2009,"Revenue":2781505847},"1":{"Release Year":1997,"Revenue":1835300000},"2":{"Release Year":2011,"Revenue":1327655619}}'

In [26]:
aa.to_json(orient="values")

'[[2009,2781505847],[1997,1835300000],[2011,1327655619]]'